<a href="https://colab.research.google.com/github/SnkhchyanV/NeuralNetworks/blob/main/RNN_imp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [59]:
class RNN():
  def __init__(self, input_size, hidden_size, output_size):
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size

    self.W_xh = np.random.normal(size=(input_size, hidden_size))
    self.W_hh = np.random.normal(size=(hidden_size, hidden_size))
    self.W_hy = np.random.normal(size=(hidden_size, output_size))
    self.b_h = np.zeros((1, hidden_size))
    self.b_y = np.zeros((1, output_size))

  def __forward(self, x_i, h_prev):
    temp =  x_i @ self.W_xh  +  h_prev @ self.W_hh + self.b_h
    h_i = np.tanh(temp)
    y_i = h_i @ self.W_hy + self.b_y
    return h_i, y_i

  def __backward(self, X, y, hidden_states):
    dW_xh = np.zeros_like(self.W_xh)
    dW_hh = np.zeros_like(self.W_hh)
    dW_hy = np.zeros_like(self.W_hy)
    db_h = np.zeros_like(self.b_h)
    db_y = np.zeros_like(self.b_y)
    dh_next = np.zeros((1, self.hidden_size))

    for i in reversed(range(X.shape[0])):
      dy = y[i]
      dy -= y[i, np.argmax(y[i])]
      dW_hy += np.outer(hidden_states[i + 1], dy)
      db_y += dy
      dh = np.dot(dy, self.W_hy.T) + dh_next
      dh_raw = (1 - hidden_states[i + 1] ** 2) * dh
      db_h += dh_raw
      dW_xh += np.outer(X[i], dh_raw)
      dW_hh += np.outer(hidden_states[i], dh_raw)
      dh_next = np.dot(dh_raw, self.W_hh.T)

    gradients = [dW_xh, dW_hh, dW_hy, db_h, db_y]
    for g in gradients:
      np.clip(g, -1, 1, out=g)

    return dW_xh, dW_hh, dW_hy, db_h, db_y


  def train(self, X, y, epochs=20, lr=0.001):
    for e in range(epochs):
      hidden_states = np.zeros((X.shape[0], X.shape[1] + 1, self.hidden_size))
      output_array = np.zeros((X.shape[0], X.shape[1], self.output_size))

      for sample in range(X.shape[0]):
        for i in range(X.shape[1]):
          hidden_states[sample, i + 1], output_array[sample, i] = self.__forward(X[sample, i], hidden_states[sample, i])

        dW_xh, dW_hh, dW_hy, db_h, db_y = self.__backward(X[sample], output_array[sample], hidden_states[sample])
        self.W_xh -= lr * dW_xh
        self.W_hh -= lr * dW_hh
        self.W_hy -= lr * dW_hy
        self.b_h -= lr * db_h
        self.b_y -= lr * db_y

    return output_array, hidden_states

  def generate(self, seed, length):
    hidden_state = np.zeros((1, self.hidden_size))
    output_sequence = []

    for _ in range(length):
      hidden_state, output = self.__forward(seed, hidden_state)
      output_sequence.append(output)
      seed = output

    return output_sequence


In [55]:
import re

string = "This book is a comprehensive introduction to text forming resources in English, along with practical procedures for analysing English texts and relating them to their contexts of use. It has been designed to complement functional grammars of English, building on the generation of discourse analysis inspired by Halliday and Hasan's Cohesion in English. The analyses presented were developed within three main theoretical and applied contexts: (i) educational linguistics (especially genre-based literacy programmes) (ii) critical linguistics (as manifested in the development of social semiotics) and (iii) computational linguistics (in dialogue with the various text generation projects based on systemic approaches to grammar and discourse). English Text's major contribution is to outline one way in which a rich semantically oriented functional grammar can be systematically related to a theory of discourse semantics, including deconstruction of contextual issues (i.e. register, genre and ideology). The chapters have been organized with the needs of undergraduate students in theoretical linguistics and postgraduate students in applied linguistics in mind."
sequences_list = re.split("[.,]", string)


m = 53  # +26 lower case characters, +26 upper case characters, and +1 for space
character = list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ ")
data = list()
labels = list()
for sequence in sequences_list:
  seq = np.array((list(sequence)))
  n = seq.size

  sequence_matrix = np.zeros((n,m))
  for i in range(n):
    for j in range(len(character)):
      if(seq[i]==character[j]):
        sequence_matrix[i][j] = 1
        break

  data.append(np.array(sequence_matrix))

  target_matrix = np.zeros((n,m))
  for i in range(n-1):
      target_matrix[i] = sequence_matrix[i+1]

  labels.append(np.array(target_matrix))


data = np.array(data, dtype='object')
labels = np.array(labels, dtype='object')

In [56]:
# print(data)
# print(labels)

In [58]:
k = 10
RNN_O = RNN(m,k,m)
# first m is input size OR every sequence size, in our situation it is onehot encoded, value 1 of each vector of encoded matrix indicates charachter in dict,
# output size is also m, it is the probability of every symbol
# k is hidden layer size, it means how many characters model will remember at each iteration, itw it is number of cells in layer
# so this model should be able to predict every next character for every 5 symbols

RNN_O.train(data, labels)
print("Nothing")

(12,)


IndexError: ignored